In [ ]:
import json
from datetime import datetime

import pandas as pd
import requests

# Binance API url
root_binance_url = 'https://api.binance.com/api/v1/klines'
symbol_url = '?symbol='
interval_url = '&interval='
start_time_url = '&startTime='
end_time_url = '&endTime='
# limit is max 500 records, max 1200 requests/minute

symbols = ['EOSBTC', 'TRXBTC', 'ONTBTC']
# intervals: 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, ...
intervals = ['5m', '15m', '30m']
end_date_time = '15.06.2018 00:00:00'


def get_data(symbol, interval):
    # from first available timestamp
    url = root_binance_url + symbol_url + symbol + interval_url + interval + start_time_url + '0'
    data = json.loads(requests.get(url).text)

    # new start time is the previous end timestamp, 500 is the limit/max
    start_time = data[-1][0]
    end_time = convert_timestamp(end_date_time)
    while start_time < end_time:
        url = root_binance_url + symbol_url + symbol + interval_url + interval + start_time_url + str(start_time)
        data_new = json.loads(requests.get(url).text)
        # omit the first element as it is equal to the last on the previous list
        data = data + data_new[1:]
        start_time = data[-1][0]

    # save it to file
    filename = 'binance_' + symbol + '_' + interval + '.json'
    with open(filename, 'w') as f:
        json.dump(data, f, sort_keys=True, indent=4, ensure_ascii=False)


def convert_timestamp(date_time):
    dt = datetime.strptime(date_time, '%d.%m.%Y %H:%M:%S')
    millis_time = dt.timestamp() * 1000

    return int(millis_time)


def prepare_data(filename):
    df = pd.read_json(filename)
    # only need [Open Time, Open, High, Low, Close, Volume] columns
    df.drop(df.columns[6:], axis=1, inplace=True)
    col_names = ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    df.columns = col_names

    # remove rows after end time
    end_time = convert_timestamp(end_date_time)
    df = df[df['Open Time'] <= end_time]

    # remove any rows with null values
    df = df.dropna()
    print(df.tail)


# for sym in symbols:
#     for t in intervals:
#         print('\n\nData for {}, interval {}'.format(sym, t))
#         get_data(sym, t)


f_name = 'binance_' + symbols[0] + '_' + intervals[2] + '.json'
prepare_data(f_name)
